<a href="https://colab.research.google.com/github/valintea/Trabajo_Tecnicas/blob/main/Trabajo_T%C3%A9cnicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
import pandas as pd
df = pd.read_csv('Tweets.csv')



In [172]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [173]:
df.airline_sentiment.value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [174]:
df['airline_sentiment'] = df['airline_sentiment'].replace({'neutral': 'positive'})

In [175]:
df.airline_sentiment.value_counts()

negative    9178
positive    5462
Name: airline_sentiment, dtype: int64

In [176]:
for tweet in df['text'][90:120]:
    print(tweet)

@VirginAmerica "You down with RNP?" "Yeah you know me!"
@VirginAmerica hi, i did not get points on my elevate account for my most recent flight, how do i add the flight and points to my account?
@VirginAmerica I like the TV and interesting video . Just disappointed in Cancelled Flightled flight when other flights went out to jfk on Saturday .
@VirginAmerica just landed in LAX, an hour after I should of been here. Your no Late Flight bag check is not business travel friendly #nomorevirgin
@VirginAmerica why is flight 345 redirected?
@VirginAmerica Is it me, or is your website down?  BTW, your new website isn't a great user experience.  Time for another redesign.
@VirginAmerica I can't check in or add a bag. Your website isn't working. I've tried both desktop and mobile http://t.co/AvyqdMpi1Y
@VirginAmerica - Let 2 scanned in passengers leave the plane than told someone to remove their bag from 1st class bin? #uncomfortable
@virginamerica What is your phone number. I can't find who to ca

In [177]:
import pandas as pd
import re


# Patrón de enlaces
patron_enlace = r'@'


# Iterar sobre las filas y aplicar la condición
for index, row in df.iterrows():
    matches += re.findall(r'@\w+', row['text'])

matches


['@dhepburn',
 '@australia',
 '@virginmedia',
 '@pacificbiznews',
 '@freddieawards',
 '@ladygaga',
 '@carrieunderwood',
 '@ladygaga',
 '@carrieunderwood',
 '@ladygaga',
 '@carrieunderwood',
 '@ladygaga',
 '@carrieunderwood',
 '@ladygaga',
 '@carrieunderwood',
 '@ladygaga',
 '@carrieunderwood',
 '@ladygaga',
 '@carrieunderwood',
 '@ladygaga',
 '@carrieunderwood',
 '@ladygaga',
 '@carrieunderwood',
 '@ladygaga',
 '@carrieunderwood',
 '@ladygaga',
 '@carrieunderwood',
 '@ladygaga',
 '@carrieunderwood',
 '@travelzoo',
 '@southwestair',
 '@jetblue',
 '@fidifamilies',
 '@chrysichrysic',
 '@reallytallchris',
 '@sfo',
 '@waltdisneyworld',
 '@jessicajaymes',
 '@dulles_airport',
 '@dallas_news',
 '@united',
 '@ttinac11',
 '@asarco_es_ar',
 '@love',
 '@united',
 '@fastcompany',
 '@faanews',
 '@newsvp',
 '@iol',
 '@lizautter',
 '@renttherunway',
 '@kciairport',
 '@virginatlantic',
 '@atwonline',
 '@gopro',
 '@nytimes',
 '@flylaxairport',
 '@capa_aviation',
 '@reuters',
 '@madbee95',
 '@freyabevan_

In [178]:
# Iterar sobre las filas y aplicar la condición
for index, row in df.iterrows():
    texto_sin_enlaces = re.sub(r'@\w+', '', row['text'])
    df.at[index, 'text'] = texto_sin_enlaces

In [179]:
for tweet in df['text'][90:120]:
    print(tweet)

 "You down with RNP?" "Yeah you know me!"
 hi, i did not get points on my elevate account for my most recent flight, how do i add the flight and points to my account?
 I like the TV and interesting video . Just disappointed in Cancelled Flightled flight when other flights went out to jfk on Saturday .
 just landed in LAX, an hour after I should of been here. Your no Late Flight bag check is not business travel friendly #nomorevirgin
 why is flight 345 redirected?
 Is it me, or is your website down?  BTW, your new website isn't a great user experience.  Time for another redesign.
 I can't check in or add a bag. Your website isn't working. I've tried both desktop and mobile http://t.co/AvyqdMpi1Y
 - Let 2 scanned in passengers leave the plane than told someone to remove their bag from 1st class bin? #uncomfortable
 What is your phone number. I can't find who to call about a flight reservation.
 is anyone doing anything there today?  Website is useless and no one is answering the phone.
 

In [180]:
import pandas as pd

# Encuentra y muestra los tweets duplicados
duplicados = df[df.duplicated(subset='text', keep=False)]
# print(duplicados['text'][0:40])
len(duplicados)
for dup in duplicados['text'][0:10]:
    print(dup)

 did you know that suicide is the second leading cause of death among teens 10-24
 Thanks!
 What is your phone number. I can't find who to call about a flight reservation.
 thanks!
 thanks so much!
 Hi! Just wanted to see if you have any new routes planned this year for Newark. Love flying you guys and hope to do so more!
  thanks!
 Thanks!
 I have 2d and 3d embossed badges and patches superior to the ones you are currently using. 
http://t.co/3fq3XElbOn
 thank you


In [181]:

import pandas as pd

duplicados_spam = duplicados[duplicados['text'].apply(len) > 30]



In [182]:
duplicados_spam['text']


10        did you know that suicide is the second leadi...
98        What is your phone number. I can't find who t...
306       Hi! Just wanted to see if you have any new ro...
366       I have 2d and 3d embossed badges and patches ...
593       What is your phone number. I can't find who t...
                               ...                        
14386     I want to speak to a human being! !!  This is...
14392     no response to DM or email yet.  customer ser...
14529                 Mad love http://t.co/4ojrSDWPkK NYC-
14543     should reconsider #usairways acquisition. Fli...
14556                      Thank you.....you do the same!!
Name: text, Length: 340, dtype: object

In [183]:
df = df[~df['tweet_id'].isin(duplicados_spam['tweet_id'])]


In [184]:
# Convertir la columna 'text' a minúsculas
df['text'] = df['text'].str.lower()
for tweet in df['text'][90:120]:
    print(tweet)

 hi, i did not get points on my elevate account for my most recent flight, how do i add the flight and points to my account?
 i like the tv and interesting video . just disappointed in cancelled flightled flight when other flights went out to jfk on saturday .
 just landed in lax, an hour after i should of been here. your no late flight bag check is not business travel friendly #nomorevirgin
 why is flight 345 redirected?
 is it me, or is your website down?  btw, your new website isn't a great user experience.  time for another redesign.
 i can't check in or add a bag. your website isn't working. i've tried both desktop and mobile http://t.co/avyqdmpi1y
 - let 2 scanned in passengers leave the plane than told someone to remove their bag from 1st class bin? #uncomfortable
 is anyone doing anything there today?  website is useless and no one is answering the phone.
 trying to add my boy prince to my ressie. sf this thursday  from lax http://t.co/gsb2j3c4gm
 why must a traveler miss a fli

<ipython-input-184-7fdaa51bcd6d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.lower()


In [185]:
import pandas as pd

# Encuentra y muestra los tweets duplicados
duplicados = df[df.duplicated(subset='text', keep=False)]
# print(duplicados['text'][0:40])
len(duplicados)
for dup in duplicados['text'][0:10]:
    print(dup)

 thanks!
 you suck!
 thanks!
 thanks so much!
 thank you!!
  thanks!
 thanks!
 thank you
 thanks
 thanks


In [186]:
#Buscamos enlaces

import pandas as pd
import re

enlaces=[]
# Iterar sobre las filas y aplicar la condición
for index, row in df.iterrows():
    enlaces += re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', row['text'])

enlaces


['https://t.co/mwpg7grezp',
 'http://t.co/ahlxhhkiyn',
 'http://t.co/ut5grrwaaa',
 'http://t.co/r8p2zy3fe4',
 'http://t.co/y7o0unxtqp',
 'http://t.co/vhp2gtdwpk',
 'http://t.co/dw5nf0ibtr',
 'http://t.co/oa2drfaoq2',
 'http://t.co/lwwdac2khx',
 'https://t.co/f2lfulcbq7',
 'http://t.co/kj1iljaebv',
 'http://t.co/hy0vrfhjht',
 'http://t.co/kek5pdmgif',
 'http://t.co/px8hqoks3r',
 'http://t.co/aptzpurop4',
 'http://t.co/avyqdmpi1y',
 'http://t.co/gsb2j3c4gm',
 'http://t.co/marcnocwzn',
 'http://t.co/tzzjhuibch',
 'http://t.co/rpdbpx3wnd',
 'http://t.co/b2xi4yg5t8',
 'http://t.co/qdljhsloi5',
 'http://t.co/glxfwp6nqh',
 'https://t.co/zv2pt6trk9',
 'http://t.co/vx9vbctdlf',
 'http://t.co/8qsqmm7kf2',
 'http://t.co/aqzwecokk2',
 'http://t.co/ewwgi97gdx',
 'http://t.co/flwmgdahxu',
 'http://t.co/ssuvwwkyhh',
 'http://t.co/rhkamx9vf5.',
 'http://t.co/pyalebgkjt',
 'http://t.co/h952rdktqy',
 'http://t.co/5b2agfd8c4',
 'http://t.co/eniqg0buzj',
 'http://t.co/vpqem31xuq',
 'http://t.co/pxdel1nq3l

In [187]:
# Brramos enlaces
# Patrón de enlaces
patron_enlace = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


# Iterar sobre las filas y aplicar la condición
for index, row in df.iterrows():
    texto_sin_enlaces = re.sub(patron_enlace, '', row['text'])
    df.at[index, 'text'] = texto_sin_enlaces


In [188]:
#Buscamos enlaces

import pandas as pd
import re

enlaces=[]
# Iterar sobre las filas y aplicar la condición
for index, row in df.iterrows():
    enlaces += re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', row['text'])

enlaces

[]

In [189]:
# Convertir la columna 'text' a minúsculas
df['text'] = df['text'].str.lower()
for tweet in df['text'][90:120]:
    print(tweet)

 hi, i did not get points on my elevate account for my most recent flight, how do i add the flight and points to my account?
 i like the tv and interesting video . just disappointed in cancelled flightled flight when other flights went out to jfk on saturday .
 just landed in lax, an hour after i should of been here. your no late flight bag check is not business travel friendly #nomorevirgin
 why is flight 345 redirected?
 is it me, or is your website down?  btw, your new website isn't a great user experience.  time for another redesign.
 i can't check in or add a bag. your website isn't working. i've tried both desktop and mobile 
 - let 2 scanned in passengers leave the plane than told someone to remove their bag from 1st class bin? #uncomfortable
 is anyone doing anything there today?  website is useless and no one is answering the phone.
 trying to add my boy prince to my ressie. sf this thursday  from lax 
 why must a traveler miss a flight to late flight check a bag?  i missed my

In [190]:
# Patrón de enlaces
patron_virgin = '@virginamerica'
patron_american ='@americanair'

for index, row in df.iterrows():
    texto_sin_enlaces = re.sub(patron_virgin, '', row['text'])
    df.at[index, 'text'] = texto_sin_enlaces
for index, row in df.iterrows():
    texto_sin_enlaces = re.sub(patron_american, '', row['text'])
    df.at[index, 'text'] = texto_sin_enlaces
# Filtrar tweets que cumplen con la condición
tweets_procesados = df['text'].loc[df['text'] != ""]

# Muestra los tweets procesados que cumplen con la condición
for tweet in tweets_procesados[90:120]:
    print(tweet)
len(tweets_procesados[90])

 hi, i did not get points on my elevate account for my most recent flight, how do i add the flight and points to my account?
 i like the tv and interesting video . just disappointed in cancelled flightled flight when other flights went out to jfk on saturday .
 just landed in lax, an hour after i should of been here. your no late flight bag check is not business travel friendly #nomorevirgin
 why is flight 345 redirected?
 is it me, or is your website down?  btw, your new website isn't a great user experience.  time for another redesign.
 i can't check in or add a bag. your website isn't working. i've tried both desktop and mobile 
 - let 2 scanned in passengers leave the plane than told someone to remove their bag from 1st class bin? #uncomfortable
 is anyone doing anything there today?  website is useless and no one is answering the phone.
 trying to add my boy prince to my ressie. sf this thursday  from lax 
 why must a traveler miss a flight to late flight check a bag?  i missed my

41

In [191]:
# Este no funciona bien lo dejo para poner porque no funciona

# Calculamos si existen duplicados
dups = df.duplicated()
# Se muestran si hay  duplicados
print(dups.any())
# Lista de las filas primeras filas duplicadas
print(df[dups]['text'])
#Numero de instancias duplicadas
df.duplicated().sum()

True
12066                           i did
12097      iphone 6 64gb (not 6 plus)
12099                 thanks so much!
12137                  great, thanks!
12138                awesome. thanks!
Name: text, dtype: object


5

In [192]:

import pandas as pd

# Supongamos que df es tu DataFrame con una columna 'text'

# Convertir la columna 'text' a minúsculas y eliminar duplicados
df_sin_duplicados = df.drop_duplicates(subset='text', keep='first', ignore_index=True)

# Muestra la diferencia entre los DataFrames original y sin duplicados
diferencia = set(df['tweet_id'])-set(df_sin_duplicados['tweet_id'])
print(diferencia)
for i in diferencia:
  print(df[df['tweet_id']==i]['text'])

print('\n duplicados con una longitud mayor de X \n')
for i in diferencia:
  if len(df[df['tweet_id']==i]['text']) >= 5:
    print(df[df['tweet_id']==i]['text'])

{567876244722814976, 567667301067915264, 568605449659895808, 569989999963213824, 567728698900488192, 568790685723136001, 570109867031191552, 569513703722393601, 568264441906618368, 567729241312489472, 568976617453658113, 569298165855338497, 567919531068391425, 569981679806365697, 570001601055883265, 570060846241611777, 567768417038761987, 569250438882705410, 568536883199746048, 568519880711872512, 568238227791822850, 569276207696453632, 568809330151022592, 568349321634013184, 567796029056385024, 568157856576475136, 569869163029762048, 568122440934658048, 569855773951561728, 567815174045868032, 568812233146404864, 567818055771918336, 568448110256640000, 568845724110213120, 568074993856335872, 570042296357539840, 570214968622239744, 568910753652199424, 570088573598355456, 568563324704706560, 570085896319401984, 568841522646552576, 569930406591664128, 568118442454355968, 568566331169247232, 570044681670696960, 568501539167473664, 569990481452539904, 568964945259466752, 569672202037170177,

In [193]:

import pandas as pd

# Supongamos que df es tu DataFrame con una columna 'text'

# Convertir la columna 'text' a minúsculas y eliminar duplicados
df_sin_duplicados = df.drop_duplicates(subset='text', keep='first', ignore_index=True)

# Muestra la diferencia entre los DataFrames original y sin duplicados
diferencia = set(df['tweet_id'])-set(df_sin_duplicados['tweet_id'])
print(diferencia)
for i in diferencia:
  print(df[df['tweet_id']==i]['text'])

  if len(texto_sin_enlaces) >= 35:

SyntaxError: ignored

In [ ]:
import pandas as pd

# Elimina duplicados basados en la columna 'text'
df_sin_duplicados = df.drop_duplicates(subset='text', keep='first')


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd


# Crear un vectorizador para contar las palabras en los textos
vectorizador = CountVectorizer()

# Aplicar el vectorizador a todo el conjunto de datos
X = vectorizador.fit_transform(df['text'])

# Obtener tablas de frecuencia segun el tipo de tweet
palabras_positivas = pd.DataFrame(X[df['airline_sentiment'] == 'positive'].toarray(), columns=vectorizador.get_feature_names_out()).sum(axis=0)
palabras_negativas = pd.DataFrame(X[df['airline_sentiment'] == 'negative'].toarray(), columns=vectorizador.get_feature_names_out()).sum(axis=0)

# Crear DataFrames para las palabras más frecuentes
palabras_positivas_contador = pd.DataFrame({'Palabra': palabras_positivas.index, 'Conteo': palabras_positivas.values})
palabras_negativas_contador = pd.DataFrame({'Palabra': palabras_negativas.index, 'Conteo': palabras_negativas.values})

# Ordenar los DataFrames por conteo en orden descendente
palabras_positivas_contador = palabras_positivas_contador.sort_values(by='Conteo', ascending=False)
palabras_negativas_contador = palabras_negativas_contador.sort_values(by='Conteo', ascending=False)

# Imprimir las palabras más frecuentes en cada clase
print("Palabras más frecuentes en tweets positivos:")
print(palabras_positivas_contador.head(10))

print("\nPalabras más frecuentes en tweets negativos:")
print(palabras_negativas_contador.head(10))


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np


# Calcular el percentil X%
percentil_positivo = np.percentile(palabras_positivas_contador['Conteo'], 99.5)
percentil_negativo = np.percentile(palabras_negativas_contador['Conteo'], 99.5)

# Identificar palabras comunes dentro del percentil
palabras_comunes_percentil = set(palabras_positivas_contador[palabras_positivas_contador['Conteo'] >= percentil_positivo]['Palabra']).intersection(
    set(palabras_negativas_contador[palabras_negativas_contador['Conteo'] >= percentil_negativo]['Palabra']))

# Eliminar palabras comunes dentro del percentil
palabras_eliminar = [palabra for palabra in palabras_comunes_percentil if
                     palabra in palabras_positivas_contador['Palabra'].values and
                     palabra in palabras_negativas_contador['Palabra'].values]

# Eliminar las palabras comunes
palabras_positivas_contador = palabras_positivas_contador[~palabras_positivas_contador['Palabra'].isin(palabras_eliminar)]
palabras_negativas_contador = palabras_negativas_contador[~palabras_negativas_contador['Palabra'].isin(palabras_eliminar)]

# Imprimir las palabras más frecuentes en cada clase después de eliminar las comunes
print("Palabras más frecuentes en tweets positivos:")
print(palabras_positivas_contador.head(10))

print("\nPalabras más frecuentes en tweets negativos:")
print(palabras_negativas_contador.head(10))

# Imprimir las palabras eliminadas
print("\nPalabras eliminadas en ambas clases dentro del percentil:")
print(palabras_eliminar)


Aquí repito lo mismo pero con traintestsplit:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np


# División en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['airline_sentiment'], test_size=0.2, random_state=42)

# Crear un vectorizador para contar las palabras en los textos
vectorizador = CountVectorizer()

# Aplicar el vectorizador a los datos de entrenamiento y prueba
X_train_vectorizado = vectorizador.fit_transform(X_train)
X_test_vectorizado = vectorizador.transform(X_test)

# Obtener las palabras más frecuentes en cada clase para los datos de entrenamiento
palabras_positivas_train = pd.DataFrame(X_train_vectorizado[y_train == 'positive'].toarray(), columns=vectorizador.get_feature_names_out()).sum(axis=0)
palabras_negativas_train = pd.DataFrame(X_train_vectorizado[y_train == 'negative'].toarray(), columns=vectorizador.get_feature_names_out()).sum(axis=0)

# Crear DataFrames para las palabras más frecuentes en cada clase para los datos de entrenamiento
palabras_positivas_contador_train = pd.DataFrame({'Palabra': palabras_positivas_train.index, 'Conteo': palabras_positivas_train.values})
palabras_negativas_contador_train = pd.DataFrame({'Palabra': palabras_negativas_train.index, 'Conteo': palabras_negativas_train.values})

# Ordenar los DataFrames por conteo en orden descendente para los datos de entrenamiento
palabras_positivas_contador_train = palabras_positivas_contador_train.sort_values(by='Conteo', ascending=False)
palabras_negativas_contador_train = palabras_negativas_contador_train.sort_values(by='Conteo', ascending=False)

# Calcular el percentil de las frecuencias para los datos de entrenamiento
percentil_positivo_train = np.percentile(palabras_positivas_contador_train['Conteo'], 99.5)
percentil_negativo_train = np.percentile(palabras_negativas_contador_train['Conteo'], 99.5)

# Identificar palabras comunes dentro del percentil para los datos de entrenamiento
palabras_comunes_percentil_train = set(palabras_positivas_contador_train[palabras_positivas_contador_train['Conteo'] >= percentil_positivo_train]['Palabra']).intersection(
    set(palabras_negativas_contador_train[palabras_negativas_contador_train['Conteo'] >= percentil_negativo_train]['Palabra']))

# Eliminar palabras comunes dentro del percentil para los datos de entrenamiento
palabras_eliminar_train = [palabra for palabra in palabras_comunes_percentil_train if
                           palabra in palabras_positivas_contador_train['Palabra'].values and
                           palabra in palabras_negativas_contador_train['Palabra'].values]

# Eliminar las palabras comunes para los datos de entrenamiento
palabras_positivas_contador_train = palabras_positivas_contador_train[~palabras_positivas_contador_train['Palabra'].isin(palabras_eliminar_train)]
palabras_negativas_contador_train = palabras_negativas_contador_train[~palabras_negativas_contador_train['Palabra'].isin(palabras_eliminar_train)]

# Imprimir las palabras más frecuentes en cada clase después de eliminar las comunes para los datos de entrenamiento
print("Palabras más frecuentes en tweets positivos (entrenamiento):")
print(palabras_positivas_contador_train.head(10))

print("\nPalabras más frecuentes en tweets negativos (entrenamiento):")
print(palabras_negativas_contador_train.head(10))

# Imprimir las palabras eliminadas para los datos de entrenamiento
print("\nPalabras eliminadas en ambas clases dentro del percentil (entrenamiento):")
print(palabras_eliminar_train)

# Aplicar el vectorizador a los datos de prueba
X_test_vectorizado = vectorizador.transform(X_test)

# # Obtener las palabras más frecuentes en cada clase para los datos de prueba
# palabras_positivas_test = pd.DataFrame(X_test_vectorizado[y_test == 'positivo'].toarray(), columns=vectorizador.get_feature_names_out()).sum(axis=0)
# palabras_negativas_test = pd.DataFrame(X_test_vectorizado[y_test == 'negativo'].toarray(), columns=vectorizador.get_feature_names_out()).sum(axis=0)

# # Crear DataFrames para las palabras más frecuentes en cada clase para los datos de prueba
# palabras_positivas_contador_test = pd.DataFrame({'Palabra': palabras_positivas_test.index, 'Conteo': palabras_positivas_test.values})
# palabras_negativas_contador_test = pd.DataFrame({'Palabra': palabras_negativas_test.index, 'Conteo': palabras_negativas_test.values})

# # Ordenar los DataFrames por conteo en orden descendente para los datos de prueba
# palabras_positivas_contador_test = palabras_positivas_contador_test.sort_values(by='Conteo', ascending=False)
# palabras_negativas_contador_test = palabras_negativas_contador_test.sort_values(by='Conteo', ascending=False)



In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Listas de palabras positivas y negativas
palabras_positivas_seleccionadas = palabras_positivas_contador_train['Palabra'].tolist()
palabras_negativas_seleccionadas = palabras_negativas_contador_train['Palabra'].tolist()

# Calcula el bonus para cada palabra
bonus_positivo = 1
bonus_negativo = -1

# Asigna puntuaciones a las palabras según su frecuencia
puntuaciones_positivas = {palabra: bonus_positivo * conteo for palabra, conteo in zip(palabras_positivas_contador_train['Palabra'], palabras_positivas_contador_train['Conteo'])}
puntuaciones_negativas = {palabra: bonus_negativo * conteo for palabra, conteo in zip(palabras_negativas_contador_train['Palabra'], palabras_negativas_contador_train['Conteo'])}
puntuaciones_totales = puntuaciones_positivas

# Combina las puntuaciones de palabras positivas y negativas
for i in puntuaciones_negativas:
  if i in puntuaciones_totales:
    puntuaciones_totales[i] += puntuaciones_negativas[i]
  else:
    puntuaciones_totales[i] = puntuaciones_negativas[i]

# Crea una función para calcular la puntuación total de un tweet
def calcular_puntuacion(tweet):
    palabras_tweet = tweet.split()
    puntuacion_tweet = sum(puntuaciones_totales.get(palabra, 0) for palabra in palabras_tweet)
    return puntuacion_tweet

# Aplica la función a cada tweet en los datos de prueba
puntuaciones_tweets = X_test.apply(calcular_puntuacion)

# Clasifica los tweets como positivos o negativos según su puntuación
tweets_clasificados = np.where(puntuaciones_tweets >= -200, 'positive', 'negative')

# Evalúa la precisión del modelo
accuracy_puntuacion = accuracy_score(y_test, tweets_clasificados)
print(f"Precisión del modelo basado en puntuaciones: {accuracy_puntuacion:.2f}")

# Muestra métricas adicionales como matriz de confusión y reporte de clasificación
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, tweets_clasificados))

print("\nReporte de Clasificación:")
print(classification_report(y_test, tweets_clasificados))


Usar un clasificador con traintest split que directamente sen entrene con los tuits vectorizados con naive bayes y por otro lado usar el recuento de palabras para hacer una votación dandole puntos a cada palabra.

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
tweets_clasificados = label_encoder.fit_transform(tweets_clasificados)

In [ ]:
from sklearn.metrics import roc_auc_score
y_test=label_encoder.fit_transform(y_test)

macro_roc_auc_ovo = roc_auc_score(y_test, tweets_clasificados)
print(macro_roc_auc_ovo)

In [ ]:
puntuaciones_totales['thanks']

In [ ]:
puntuaciones_tweets.sort_values()

In [ ]:
print(y_test)
print(tweets_clasificados)

In [ ]:
tweets_clasificados